#### This Version Creates the Graph in Neo4J then runs the Queries.

In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

In [4]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_2_1')

# Define the destination directory
# This should be the import directory of your graph database
destination_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-0898892c-35f2-4a77-9759-d6f57a986ee9/import'

# Specify the filenames to move
filenames = ['nodes.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [5]:
# Define the working directory
working_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-0898892c-35f2-4a77-9759-d6f57a986ee9/bin'

# Construct the command
#command = [
#    './bin/neo4j-admin', 'database', 'import', 'full',
#    '--nodes=import/nodes.csv',
#    '--relationships=import/relationships.csv', 'neo4j'
#]

command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv neo4j'

# Execute the command


result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Error during import:
WARNING! You are using an unsupported Java runtime.
* Please use Oracle(R) Java(TM) 17, OpenJDK(TM) 17 to run Neo4j.
* Please see https://neo4j.com/docs/ for Neo4j installation instructions.
org.neo4j.internal.batchimport.input.HeaderException: Group 'null' not found. Available groups are: []
	at org.neo4j.internal.batchimport.input.Groups.get(Groups.java:77)
	at org.neo4j.internal.batchimport.input.csv.DataFactories$DefaultRelationshipFileHeaderParser.create(DataFactories.java:445)
	at org.neo4j.internal.batchimport.input.csv.DataFactories.parseHeaderEntries(DataFactories.java:196)
	at org.neo4j.internal.batchimport.input.csv.DataFactories$AbstractDefaultFileHeaderParser.create(DataFactories.java:221)
	at org.neo4j.internal.batchimport.input.csv.CsvInput.verifyHeaders(CsvInput.java:208)
	at org.neo4j.internal.batchimport.input.csv.CsvInput.<init>(CsvInput.java:145)
	at org.neo4j.internal.batchimport.input.csv.CsvInput.<init>(CsvInput.java:108)
	at org.neo4j.import

: 

Run and connect to the Neo4j Database

In [8]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [9]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      'Spectra',
      {
        SIMILAR: {
          orientation: 'UNDIRECTED'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Define the graph algorithms.

In [54]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [55]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [57]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [59]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph')
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [60]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph')
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [61]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph')
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [62]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph')
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [51]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        {randomSeed:1234,
         embeddingDimension: 256,
         iterationWeights: [0.0, 0.0, 1.0, 1.0]
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name,  embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [63]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [64]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'Louvain'])

In [65]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, label_propagation_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on='name', how='left')

In [66]:
df = merged_df.rename(columns={'name' : 'SpecID'})
df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,LabelPropagation,Leiden,Louvain
0,210224-1-34,1.397131,868.0,0.038603,0.334618,16,24,2415
1,210414-1-33,1.370523,850.0,0.037945,0.330898,16,24,2415
2,210224-1-38,1.358143,850.0,0.038002,0.330713,16,24,2415
3,210428-3-31,1.357280,841.0,0.037663,0.328811,16,24,2415
4,210428-2-14,1.351905,844.0,0.037591,0.329392,16,24,2415
...,...,...,...,...,...,...,...,...
3040,210225-2-16,0.166230,6.0,0.000153,0.151369,16,21,467
3041,210325-2-35,0.165969,8.0,0.000306,0.151757,16,24,2415
3042,210407-2-29,0.157303,5.0,0.000167,0.151067,16,24,2415
3043,210407-1-01,0.154512,3.0,0.000065,0.150647,16,0,440


In [39]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_246,embedding_247,embedding_248,embedding_249,embedding_250,embedding_251,embedding_252,embedding_253,embedding_254,embedding_255
0,201210-1-00,0.035269,-0.037547,0.031784,-0.027910,0.030410,0.180765,0.104346,0.024844,-0.004462,...,0.091175,-0.173984,0.151440,-0.164248,0.067395,-0.130668,0.033660,-0.013763,0.044749,0.027484
1,201210-1-01,0.038106,-0.039755,0.027535,-0.029440,0.025530,0.181263,0.105157,0.025292,-0.006119,...,0.095302,-0.175739,0.147734,-0.164330,0.065153,-0.134692,0.035600,-0.016797,0.045062,0.025099
2,201210-1-02,-0.029393,0.015198,0.126618,0.053306,0.099745,0.106525,0.062500,0.017099,0.070272,...,-0.019153,-0.104287,0.194545,-0.130098,0.072441,-0.008741,-0.045183,0.062151,0.038004,0.084373
3,201210-1-03,-0.050117,0.088726,0.162988,0.055606,0.131341,0.027026,-0.019304,0.016400,0.125736,...,-0.102769,-0.046732,0.225308,-0.112851,0.069677,0.079977,-0.092380,0.101936,0.013405,0.091302
4,201210-1-04,-0.116077,-0.055523,0.156792,0.110959,0.164191,0.054516,0.064967,-0.024916,0.003023,...,-0.080449,-0.012806,0.107279,0.019336,0.065371,0.049116,-0.021362,0.030219,0.073889,0.053334


Delete the projection

In [17]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)

In [18]:
close_driver()

In [19]:
df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,LabelPropagation,Leiden,Louvain
0,210519-3-01,1.522488,18235.0,0.029951,0.659144,1,2548,1160
1,210217-2-11,1.504415,18017.0,0.029593,0.652600,1,2548,1160
2,210217-2-05,1.471457,17589.0,0.028890,0.640493,1,2548,1160
3,210217-2-04,1.471432,17591.0,0.028893,0.640519,1,2548,1160
4,210217-2-18,1.469541,17574.0,0.028865,0.639833,1,2548,1160
...,...,...,...,...,...,...,...,...
3040,210519-2-37,0.417426,3510.0,0.005765,0.248813,1,2548,1160
3041,210114-1-17,0.399481,3243.0,0.005327,0.241883,1,1,2299
3042,210304-2-20,0.397664,3213.0,0.005277,0.241136,1,1,2299
3043,210211-2-27,0.386780,3080.0,0.005059,0.237206,1,1,2299


In [13]:
status_df = pd.read_csv('../../data/graph_2_1/status.csv')

In [14]:
status_df.drop(columns=['Unnamed: 0'], inplace=True)

In [18]:
status_df

,SpecID,Status
0,201210-1-00,Normal
1,201210-1-01,Normal
2,201210-1-02,Normal
3,201210-1-03,Normal
4,201210-1-04,Normal
...,...,...
3040,210526-3-45,Hyperglycemia
3041,210526-3-46,Hyperglycemia
3042,210526-3-47,Hyperglycemia
3043,210526-3-48,Hyperglycemia


In [40]:
graph_df = pd.merge(fastRP_df, status_df, left_on="SpecID", right_on="SpecID", how="inner")

In [67]:
graph_df = pd.merge(df, status_df, left_on="SpecID", right_on="SpecID", how="inner")

In [68]:
graph_df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,LabelPropagation,Leiden,Louvain,Status
0,210224-1-34,1.397131,868.0,0.038603,0.334618,16,24,2415,Hypoglycemia
1,210414-1-33,1.370523,850.0,0.037945,0.330898,16,24,2415,Normal
2,210224-1-38,1.358143,850.0,0.038002,0.330713,16,24,2415,Hypoglycemia
3,210428-3-31,1.357280,841.0,0.037663,0.328811,16,24,2415,Normal
4,210428-2-14,1.351905,844.0,0.037591,0.329392,16,24,2415,Normal
...,...,...,...,...,...,...,...,...,...
3040,210225-2-16,0.166230,6.0,0.000153,0.151369,16,21,467,Hypoglycemia
3041,210325-2-35,0.165969,8.0,0.000306,0.151757,16,24,2415,Normal
3042,210407-2-29,0.157303,5.0,0.000167,0.151067,16,24,2415,Normal
3043,210407-1-01,0.154512,3.0,0.000065,0.150647,16,0,440,Hypoglycemia


In [42]:
def modeler(k_folds=5, model='linear', show_matrix=True):

    acc_scores = []

    # Drop the SpecID column
    X = graph_df.drop(columns=['SpecID', 'Status'])

    # Target variable
    y = graph_df['Status']

    for i in range(0, k_folds):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
        clf = svm.SVC(kernel='linear', class_weight='balanced')
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        acc = accuracy_score(pred, y_test)
        acc_scores.append(acc)        
        
    print('Accuracy scores: ', acc_scores)
    print('Mean accuracy: ', np.mean(acc_scores))
    
    # if show_matrix:
    #     matrix = plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues, normalize='true')
    #     plt.show(matrix)
    #     plt.show()
        
    return

In [43]:
modeler()

Accuracy scores:  [0.4540682414698163, 0.431758530183727, 0.4658792650918635, 0.4343832020997375, 0.44881889763779526]
Mean accuracy:  0.4469816272965879


In [69]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = graph_df.drop(columns=['SpecID', 'Status'])

# Target variable
y = graph_df['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)


Random Forest Accuracy: 0.48111658456486045
Extra Trees Accuracy: 0.4745484400656814


In [26]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [27]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(10)

    return top_10_features

In [28]:
calculate_metrics(y_test, rf_predictions)

Overall Accuracy: 0.555008210180624


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.56      0.51      0.53       203
 Hypoglycemia       0.50      0.57      0.54       198
       Normal       0.61      0.58      0.60       208

     accuracy                           0.56       609
    macro avg       0.56      0.55      0.55       609
 weighted avg       0.56      0.56      0.56       609


Confusion Matrix:
[[104  61  38]
 [ 46 113  39]
 [ 37  50 121]]


In [29]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.5517241379310345


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.54      0.50      0.52       203
 Hypoglycemia       0.51      0.56      0.54       198
       Normal       0.60      0.60      0.60       208

     accuracy                           0.55       609
    macro avg       0.55      0.55      0.55       609
 weighted avg       0.55      0.55      0.55       609


Confusion Matrix:
[[101  62  40]
 [ 45 111  42]
 [ 41  43 124]]


In [30]:
get_feature_importances(rf_model, X)

,Feature,Importance
245,embedding_245,0.007585
223,embedding_223,0.007002
172,embedding_172,0.006364
230,embedding_230,0.006254
195,embedding_195,0.006153
97,embedding_97,0.006072
47,embedding_47,0.006011
213,embedding_213,0.005891
107,embedding_107,0.005864
78,embedding_78,0.005863


In [31]:
get_feature_importances(et_model, X)

,Feature,Importance
168,embedding_168,0.006789
170,embedding_170,0.006593
186,embedding_186,0.005763
9,embedding_9,0.005476
180,embedding_180,0.005362
74,embedding_74,0.005287
7,embedding_7,0.005271
166,embedding_166,0.005194
134,embedding_134,0.005040
242,embedding_242,0.004967
